# Capstone Hotel reservation prediction

![Logo](img/hotel-logo.png)

## Modelling Repeater
#### (Folgebuchung)

### Import required libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

#modelling
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
#from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict, RandomizedSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

#visualisations
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%matplotlib inline

#### Loading cleaned dataset

In [2]:
df = pd.read_csv("data/dataset_w_feature.csv", index_col=0, encoding="iso-8859-15")

In [3]:
df["buchungsdatum"]      = pd.to_datetime(df["buchungsdatum"])
df["anreisedatum"]       = pd.to_datetime(df["anreisedatum"])
df["abreisedatum"]       = pd.to_datetime(df["abreisedatum"])

In [4]:
df.head()

,buchungsdatum,anreisedatum,abreisedatum,naechte_insgesamt,gebuchte_naechte,personen_anzahl,gesamtpreis,saison,buchung_telefon,buchung_online,buchung_reisebuero,zimmerarten,reise_adventure,reise_relax,reise_standard,reise_genuss,kind,entfernung,zahler,kunden_id,buchungsnr,buchungsnr_reisebuero,menue_code,zusatzleistung,destination,anrede,plz,ort,kundenclub,jahrgangsalter_klassiert,bundesland,einwohner_adjusted_plz,bev_m_plz,bev_w_plz,bev_insgesamt_ort,bev_weiblich_ort,bev_maennlich_ort,qkm_plz,flaeche_ort,einwohnerdichte_prokm2_plz,haushalte_regiog_2017_plz,kaufkraft_einwohner_in_eur_regio,kk_einwohner_index_regiog_2017,kk_hh_index_regiog_2017,gemeindetyp_regiostar_2016,wirtschaftskraftmr1,wirtschaftskraftmr2,buchungsjahr,buchungsquartal,buchungsmonat,buchungswoche,buchungstag,buchung_wochentag,anreisejahr,anreisequartal,anreisemonat,anreisewoche,anreisetag,anreise_wochentag,abreisejahr,abreisequartal,abreisemonat,abreisewoche,abreisetag,abreise_wochentag,buchungsmonat-jahr,buchungsquartal-jahr,anreisemonat-jahr,anreisequartal-jahr,abreisemonat-jahr,abreisequartal-jahr,buchungskanal,werbe_ban,anz_werbe_ban,aufenthalt,naechte_ausserhalb,anz_buch,buchungsvorlauf,preis_per,preis_nacht,preis_nacht_per,VIP,buch_zeilen,erstbuchung,folgebuchung,repeater
0,2010-01-29,2010-04-02,2010-04-10,8,8,2,2982,2010,0,0,1,classic,0,0,0,1,0,535.48,1,1,1151,456,5,5,linz,Herr,29664,Walsrode,0,55-64,Niedersachsen,25173,12540,12633,25173,11577,11491,344.326464,344.326464,73.108,11453,20641.89,92.819,98.104,Landregion - Zentrale Stadt,-0.333392,-0.388738,2010,1,1,4,4,Freitag,2010,2,4,13,4,Freitag,2010,2,4,14,5,Samstag,20101,20101,20104,20102,20104,20102,reisebuero,keine,0,8,0,1,63,1491.00,372.75,186.38,0,2,1,0,0
1,2010-01-14,2010-04-05,2010-04-10,5,5,2,1437,2010,0,0,1,classic,0,0,1,0,0,544.72,0,2,40,6413,5,5,linz,Frau,50226,Frechen,0,55-64,Nordrhein-Westfalen,52473,25645,26828,52473,26828,25645,45.118960,45.118960,1162.992,26227,25268.58,113.623,111.379,Landregion - Zentrale Stadt,0.853802,-0.345112,2010,1,1,2,3,Donnerstag,2010,2,4,14,0,Montag,2010,2,4,14,5,Samstag,20101,20101,20104,20102,20104,20102,reisebuero,generell,5,5,0,1,81,718.50,287.40,143.70,0,2,1,0,0
2,2010-03-01,2010-04-05,2010-04-10,5,5,2,1737,2010,0,0,1,classic,0,0,1,0,0,606.63,0,3,2134,4322,5,5,linz,Herr,23970,Wismar,0,75-84,Mecklenburg-Vorpommern,1423,692,731,46442,21844,20706,50.563027,83.131346,17.117,6355,18697.65,84.076,73.665,"Stadtregion - Kleinstädtischer, dörflicher Raum",-1.012536,-1.012536,2010,1,3,9,0,Montag,2010,2,4,14,0,Montag,2010,2,4,14,5,Samstag,20103,20101,20104,20102,20104,20102,reisebuero,keine,0,5,0,1,35,868.50,347.40,173.70,0,2,1,0,0
3,2010-01-25,2010-03-21,2010-03-25,4,4,2,1047,2010,0,0,1,classic,0,1,0,0,0,421.59,1,4,776,6414,69,9,linz,Frau,15732,Eichwalde,0,65-74,Brandenburg,14690,7302,7388,14690,4135,4087,11.965427,11.965427,1227.704,7159,23442.06,105.410,104.087,Landregion - Städtischer Raum,0.417226,0.417226,2010,1,1,4,0,Montag,2010,1,3,11,6,Sonntag,2010,1,3,12,3,Donnerstag,20101,20101,20103,20101,20103,20101,reisebuero,keine,0,4,0,1,55,523.50,261.75,130.88,0,1,1,0,0
4,2010-01-26,2010-03-07,2010-03-09,2,2,20,5279,2010,0,0,1,classic,1,0,0,0,0,37.88,0,5,813,2938,113,2,duesseldorf,Frau,53881,Euskirchen,1,45-54,Nordrhein-Westfalen,28531,13975,14556,57975,29578,28397,127.679129,139.430913,204.625,11683,23328.82,104.901,120.110,"Stadtregion - Kleinstädtischer, dörflicher Raum",0.212335,-0.579173,2010,1,1,4,1,Dienstag,2010,1,3,9,6,Sonntag,2010,1,3,10,1,Dienstag,20101,20101,20103,20101,20103,20101,reisebuero,keine,0,2,0,1,40,263.95,2639.50,131.98,0,7,1,0,1


## Preparation

#### Dropping redundant variables

In [5]:
df.drop("buchungskanal", axis=1, inplace=True)
df.drop("werbe_ban", axis=1, inplace=True)

#### Grouping variables

In [6]:
categorial = list(df.columns[df.dtypes==object])
datetime64 = ['buchungsdatum', 'anreisedatum', 'abreisedatum']
numeric    = list(df.select_dtypes(include=['int64', 'float64']).columns)

for entry in ['erstbuchung', 'repeater', 'kunden_id', 'folgebuchung', 'menue_code', 'zusatzleistung', 'plz', 'kundenclub', 'anz_buch']: 
    numeric.remove(entry)
    
for entry in ['menue_code', 'zusatzleistung']: 
    categorial.append(entry)    

categorial.remove('ort')

# 7 = 'erstbuchung', 'repeater', 'kunden_id', 'folgebuchung', 'plz', 'kundenclub', 'ort'
df.shape[1] == len(categorial) + len(datetime64) + len(numeric) + 7

False

#### renaming destinations

In [7]:
aenderung = { "saint etienne": "saint-etienne",
              "viana do castelo": "viana-do-castelo",
            }

df.destination.replace(aenderung, inplace=True)

aenderung2 = {
"Landregion - Kleinstädtischer, dörflicher Raum": "Kleinstädt_(Land)",
"Landregion - Städtischer Raum": "Stadt_(Land)",
"Landregion - Zentrale Stadt": "zen_Stadt_(Land)",
"Stadtregion - Kleinstädtischer, dörflicher Raum": "Kleinstädt_(Stadt)",
"Stadtregion - Metropole": "Metropole",
"Stadtregion - Mittelstadt, städtischer Raum": "Mittelstadt",
"Stadtregion -Regiopole und Großstadt": "Regiopole"
    }

df.gemeindetyp_regiostar_2016.replace(aenderung2, inplace=True)

In [8]:
df.menue_code     = df.menue_code.astype("str")
df.zusatzleistung = df.zusatzleistung.astype("str")

### Normalisation and dummy encoding

In [9]:
df = df.reset_index()
x = df[numeric]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
columns = df[numeric].columns
df_num = pd.DataFrame(x_scaled)
df_num.columns = columns

df_dum = pd.get_dummies(df[categorial])
df_dum.shape

(198114, 291)

## Logistic Regression
Basic (only numeric features)

In [10]:
X = df[numeric]
y = df.folgebuchung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

lr1 = LogisticRegression(solver='liblinear', max_iter=10_000)

scoring = ['precision', 'recall', 'f1', 'roc_auc']
scores1 = cross_validate(lr1, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)
# cv can also return train score but we set it to false

y_pred1 = cross_val_predict(lr1, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   48.0s remaining:  1.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   51.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   42.5s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   48.3s finished


only numeric features but normalised

In [11]:
X = df_num
y = df.folgebuchung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

lr2 = LogisticRegression(solver='liblinear', max_iter=10_000)

scores2 = cross_validate(lr2, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)
# cv can also return train score but we set it to false

y_pred2 = cross_val_predict(lr2, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   27.4s remaining:   41.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   30.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   23.2s remaining:   34.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   28.6s finished


### extended Logistic Regression
(categorial variables as dummies)

In [12]:
X = pd.concat([df_num, df_dum], axis=1)
y = df.folgebuchung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

lr3 = LogisticRegression(solver='liblinear', max_iter=10_000)

scores3 = cross_validate(lr3, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred3 = cross_val_predict(lr3, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   45.9s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   54.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   43.5s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   53.0s finished


### Dealing with imbalace 
in target feature folgebuchung by resampling

In [13]:
df.folgebuchung.value_counts()

0    158478
1     39636
Name: folgebuchung, dtype: int64

New train/test split without normalisation

In [14]:
X = pd.concat([df[numeric], df_dum], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

#### Upsampling

In [15]:
# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
erstbuchung  = X[X.folgebuchung==0]
folgebuchung = X[X.folgebuchung==1]

# upsample minority
folgebuchung_upsampled = resample(folgebuchung, replace=True, 
                          n_samples=len(erstbuchung), # match number in majority class
                          random_state=1337) 

# combine majority and upsampled minority
upsampled = pd.concat([erstbuchung, folgebuchung_upsampled])

upsampled.folgebuchung.value_counts()

1    142745
0    142745
Name: folgebuchung, dtype: int64

#### Downsampling

In [16]:
# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
erstbuchung  = X[X.folgebuchung==0]
folgebuchung = X[X.folgebuchung==1]

# downsample majority
erstbuchung_downsampled = resample(erstbuchung, replace=False, 
                          n_samples=len(folgebuchung), # match minority n
                          random_state=1337) 

# combine majority and upsampled minority
downsampled = pd.concat([folgebuchung, erstbuchung_downsampled])

downsampled.folgebuchung.value_counts()

1    35557
0    35557
Name: folgebuchung, dtype: int64

### balanced Logistic Regression
with downsampled data and additional dummy variables

In [17]:
y_train = upsampled.folgebuchung
X_train = upsampled.drop('folgebuchung', axis=1)

lr4 = LogisticRegression(solver='liblinear', max_iter=10_000, n_jobs=-1, verbose=5)

scores4 = cross_validate(lr4, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred4 = cross_val_predict(lr4, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  5.3min remaining:  8.0min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  7.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.9min remaining:  7.4min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  7.2min finished


### balanced Logistic Regression
downsampled

In [18]:
y_train = downsampled.folgebuchung
X_train = downsampled.drop('folgebuchung', axis=1)

lr5 = LogisticRegression(solver='liblinear', max_iter=10_000, n_jobs=-1, verbose=5)

scores5 = cross_validate(lr5, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred5 = cross_val_predict(lr5, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.6min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.8min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.3min finished


# Decision Trees
## Random forest
with manual search

In [19]:
X = pd.concat([df[numeric], df_dum], axis=1)
y = df.folgebuchung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

lr6 = RandomForestClassifier(max_depth=5, n_estimators=7, random_state=1337)

scores6 = cross_validate(lr6, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred6 = cross_val_predict(lr6, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

# y_pred5 = lr5.predict(X_test) #raus aber warum X_test

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.1s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    5.0s remaining:    7.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.7s finished


### with random search

In [ ]:
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(3, 6, 11, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(5, 120, 15, dtype = int))}

lr7 = RandomForestClassifier()
scores7 = RandomizedSearchCV(estimator = lr7, param_distributions = random_search, 
                             n_iter = 40, cv = 5, verbose= 5, random_state= 1337, n_jobs = -1)

scores7.fit(X_train,y_train)

y_pred7 = cross_val_predict(lr7, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   39.5s


## Advanced models
### GaussianNB

In [ ]:
X = pd.concat([df[numeric], df_dum], axis=1)
y = df.folgebuchung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

lr8 = GaussianNB()

scores8 = cross_validate(lr8, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred8 = cross_val_predict(lr8, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

### DecisionTreeClassifier

In [ ]:
lr9 = DecisionTreeClassifier(random_state=1337)

scores9 = cross_validate(lr9, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred9 = cross_val_predict(lr9, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

### KNeighborsClassifier

In [ ]:
lr10 = KNeighborsClassifier(n_neighbors=3)

scores10 = cross_validate(lr10, X_train, y_train, cv=5, scoring=scoring, return_train_score=False, n_jobs=-1, verbose=5)

y_pred10 = cross_val_predict(lr10, X_train, y_train, cv=5, n_jobs=-1, verbose=5)

## Model comparsion

#### Confusion Matrix & Classification Report
<img src="img/conf_matrix.png" alt="Drawing" style="width: 400px;"/>

In [ ]:
print(f"Model1:\n{confusion_matrix(y_train,y_pred1)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred1)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred1)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred1)*100,2)}")

print(f"\n\nModel2:\n{confusion_matrix(y_train,y_pred2)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred2)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred2)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred2)*100,2)}")

y_train = upsampled.folgebuchung
print(f"\n\nModel3:\n{confusion_matrix(y_train,y_pred3)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred3)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred3)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred3)*100,2)}")

y_train = downsampled.folgebuchung
print(f"\n\nModel4:\n{confusion_matrix(y_train,y_pred4)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred4)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred4)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred4)*100,2)}")

X = pd.concat([df[numeric], df_dum], axis=1)
y = df.folgebuchung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)
print(f"\n\nModel5:\n{confusion_matrix(y_train,y_pred5)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred5)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred5)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred5)*100,2)}")

print(f"\n\nModel6:\n{confusion_matrix(y_train,y_pred6)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred6)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred6)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred6)*100,2)}")

print(f"\n\nModel7:\n{confusion_matrix(y_train,y_pred7)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred7)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred7)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred7)*100,2)}")

print(f"\n\nModel8:\n{confusion_matrix(y_train,y_pred8)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred8)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred8)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred8)*100,2)}")

print(f"\n\nModel9:\n{confusion_matrix(y_train,y_pred9)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred9)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred9)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred9)*100,2)}")

print(f"\n\nModel10:\n{confusion_matrix(y_train,y_pred10)}")
print(f"\nAccuracy:{round(accuracy_score(y_train, y_pred10)*100,2)}")
print(f"Precision_score:{round(precision_score(y_train, y_pred10)*100,2)}")
print(f"Recall:{round(recall_score(y_train, y_pred8)*100,2)}")

## ROC AUC

In [ ]:
## SVM / k-nearest neighbor

In [ ]:
stop

In [ ]:
predictionforest = model.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc3 = accuracy_score(y_test,predictionforest)

In [ ]:
model.best_score_

In [ ]:
table = pd.pivot_table(pd.DataFrame(model.cv_results_),
    values='mean_test_score', index='param_n_estimators', 
                       columns='param_criterion')
     
sns.heatmap(table)

In [ ]:
model.best_params_

a. max_leaves (here: 6)
b. max_depth (here: 3)
c. min_sample_split (here: 400)
d. max_features (here: 6) - not always helpful

Feature Importance

In [ ]:
#clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

Receiver Operating Statistic (ROC) and the Area Under the Curve (AUC)

In [ ]:
stop

fpr_RF, tpr_RF, thresholds_RF = roc_curve(df.actual_label.values, df.model_RF.values)
fpr_LR, tpr_LR, thresholds_LR = roc_curve(df.actual_label.values, df.model_LR.values)

plt.plot(fpr_RF, tpr_RF,'r-',label = 'RF')
plt.plot(fpr_LR,tpr_LR,'b-', label= 'LR')
plt.plot([0,1],[0,1],'k-',label='random')
plt.plot([0,0,1,1],[0,1,1,1],'g-',label='perfect')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
#https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter

#sklearn.pipeline.Pipeline 

In [ ]:
#df_dum.head(2)

In [ ]:
#categorial

In [ ]:
conf_matrix1 = confusion_matrix(y_test, y_pred1)
class_rep1 = classification_report(y_test, y_pred1)
acc1 = accuracy_score(y_test, y_pred)*100
pre1 = precision_score(y_test, y_pred)*100
rec1 = recall_score(y_test, y_pred)*100
f1-1 = f1_score(y_test, y_pred)*100
score1

In [ ]:
confusion_matrix(y_train, y_pred1)

In [ ]:
print(classification_report(y_train, y_pred1))

In [ ]:
print(confusion_matrix(y_train,y_pred3))
print(classification_report(y_train,y_pred3))
print(accuracy_score(y_train,y_pred3))